# 앙상블 (Ensemble)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import warnings
warnings.filterwarnings('ignore')

### Voting
- hard voting: 여러 개의 예측치에 대해 다수결로 결정
- soft voting: 여러 개의 예측 확률을 평균내어 결정

##### 위스콘신 유방암 데이터셋 (Wisconsin Breast Cancer Dataset)

유방암의 악성(Malignant)과 양성(Benign)을 분류하기 위해 자주 사용되는 데이터셋
(의학적인 이미지를 바탕으로 유방암 종양의 특징을 수치화한 데이터)

**데이터셋 개요**
- **목적**: 유방암 종양이 악성(Malignant)인지, 양성(Benign)인지 분류
- **샘플 수**: 569개
- **특징(Features) 수**: 30개 (==30개의 컬럼)
- **타겟(Target)**: 0(악성) 또는 1(양성)

**데이터 구성**
1. **Radius mean**: 종양의 평균 반지름
2. **Texture mean**: 종양의 표면의 거칠기
3. **Perimeter mean**: 종양의 평균 둘레 길이
4. **Area mean**: 종양의 평균 면적
5. **Smoothness mean**: 종양의 매끄러움 정도
6. **Compactness mean**: 종양의 압축도
7. **Concavity mean**: 종양의 오목함
8. **Concave points mean**: 종양의 오목한 점 개수
9. **Symmetry mean**: 종양의 대칭성
10. **Fractal dimension mean**: 종양의 프랙탈 차원 

---

### load data

In [6]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
# data.keys() #['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module']
# print(data.DESCR)

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [76]:
#  데이터 준비 (분리)
from sklearn.model_selection import train_test_split

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((426, 30), (143, 30), (426,), (143,))

---

##### hard voting

In [62]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# 모델 객체 생성
knn_clf = KNeighborsClassifier()
lr_clf = LogisticRegression()
dt_clf = DecisionTreeClassifier()

voting_clf = VotingClassifier(
    estimators=[
        ('knn_clf', knn_clf),
        ('lr_clf', lr_clf),
        ('dt_clf', dt_clf)
    ], # 투표에 참여할 모델들
    voting='hard' # default 값
)

# 앙상블 모델 학습
voting_clf.fit(X_train, y_train)

# 예측 및 평가
y_pred_train = voting_clf.predict(X_train)
acc_score_train = accuracy_score(y_train, y_pred_train) # 실제값과 예측값 넣어 평가
print('학습 점수: ', acc_score_train)

y_pred_test = voting_clf.predict(X_test)
acc_score_test = accuracy_score(y_test, y_pred_test) # 실제값과 예측값 넣어 평가
print('테스트 점수: ', acc_score_test)

학습 점수:  0.9647887323943662
테스트 점수:  0.958041958041958


In [63]:
# hard voting 작동 원리 == 다수결
start, end = 40, 50
voting_clf_pred = voting_clf.predict(X_test[start:end])
print('앙상블 예측값: ', voting_clf_pred)

for classifier in [knn_clf, lr_clf, dt_clf]:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    acc_score = accuracy_score(y_test, pred)

    class_name = classifier.__class__.__name__ # 객체를 중심으로 해서 클래스의 이름 속성을
    print('---------------')
    print(f'{class_name} 개별 정확도: {acc_score: .4f}')
    print(f'{class_name} 예측값: {pred[start:end]}')

앙상블 예측값:  [0 1 0 1 0 0 1 1 1 0]
---------------
KNeighborsClassifier 개별 정확도:  0.9371
KNeighborsClassifier 예측값: [0 1 0 1 0 0 1 1 1 0]
---------------
LogisticRegression 개별 정확도:  0.9441
LogisticRegression 예측값: [0 1 0 1 0 0 1 1 1 0]
---------------
DecisionTreeClassifier 개별 정확도:  0.8671
DecisionTreeClassifier 예측값: [0 1 0 1 0 0 1 0 1 0]


##### soft voting

In [64]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# 모델 객체 생성
knn_clf = KNeighborsClassifier()
lr_clf = LogisticRegression()
dt_clf = DecisionTreeClassifier(random_state=0) # 설정 안하면 test 점수 바뀜뀜

voting_clf = VotingClassifier(
    estimators=[
        ('knn_clf', knn_clf),
        ('lr_clf', lr_clf),
        ('dt_clf', dt_clf)
    ], # 투표에 참여할 모델들
    voting='soft' # default 값
)

# 앙상블 모델 학습
voting_clf.fit(X_train, y_train)

# 예측 및 평가
y_pred_train = voting_clf.predict(X_train)
acc_score_train = accuracy_score(y_train, y_pred_train) # 실제값과 예측값 넣어 평가
print('학습 점수: ', acc_score_train)

y_pred_test = voting_clf.predict(X_test)
acc_score_test = accuracy_score(y_test, y_pred_test) # 실제값과 예측값 넣어 평가
print('테스트 점수: ', acc_score_test)

학습 점수:  0.9859154929577465
테스트 점수:  0.9370629370629371


In [ ]:
# soft voting 작동 원리 == 각 예측기의 확률값 평균
# 모델이 계산한 값과 직접 평균 계산한 값이 같은지 확인하기
start, end = 40, 50

voting_clf_pred_proba = voting_clf.predict_proba(X_test[start:end])
print('앙상블 예측값: ', voting_clf_pred_proba)

averages = np.full_like(voting_clf_pred_proba, 0)

for classifier in [knn_clf, lr_clf, dt_clf]:
    # 개별 학습 및 예측
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    acc_score = accuracy_score(y_test, pred)
    pred_proba = classifier.predict_proba(X_test[start:end])

    # 예측 확률 평균을 위한 합계
    averages += pred_proba

    class_name = classifier.__class__.__name__ # 객체를 중심으로 해서 클래스의 이름 속성을
    # print('---------------')
    # print(f'{class_name} 개별 정확도: {acc_score: .4f}')
    # print(f'{class_name} 예측 확률: {pred_proba}')

# 예측 확률 평균 계산 및 출력
calc_averages = averages / 3
print('각 모델별 예측값의 평균:', calc_averages)
print(np.array_equal(voting_clf_pred_proba, calc_averages)) # True -> 각 데이터가 같은 위치에 같은 값을 가지고 있는지 확인해주는 함수:array_equal

앙상블 예측값:  [[5.70263157e-01 4.29736843e-01]
 [1.08113730e-03 9.98918863e-01]
 [9.99622506e-01 3.77494355e-04]
 [3.35757426e-04 9.99664243e-01]
 [9.00993416e-01 9.90065841e-02]
 [1.00000000e+00 1.75163138e-13]
 [7.79971341e-05 9.99922003e-01]
 [1.83004552e-02 9.81699545e-01]
 [1.14568790e-03 9.98854312e-01]
 [9.32982089e-01 6.70179112e-02]]
각 모델별 예측값의 평균: [[5.70263157e-01 4.29736843e-01]
 [1.08113730e-03 9.98918863e-01]
 [9.99622506e-01 3.77494355e-04]
 [3.35757426e-04 9.99664243e-01]
 [9.00993416e-01 9.90065841e-02]
 [1.00000000e+00 1.75163138e-13]
 [7.79971341e-05 9.99922003e-01]
 [1.83004552e-02 9.81699545e-01]
 [1.14568790e-03 9.98854312e-01]
 [9.32982089e-01 6.70179112e-02]]
True


---

### Bagging
- Bootstrap Aggregation
- Bootstrap 방식의 샘플링: 각 estimator마다 훈련 데이터를 뽑을 때, 중복값을 허용하는 방식
- 분류 모델의 경우, 각 tree(estimator)의 예측값을 다수결(hard voting) 결정
- 회귀 모델의 경우, 각 tree(estimator)의 예측값을 평균내어 결정
- 기본적으로 100개의 tree 사용

**하이퍼 파라미터**
| **하이퍼파라미터**      | **설명**                                                                                     | **기본값**      |
|--------------------------|--------------------------------------------------------------------------------------------|-----------------|
| `n_estimators`           | 생성할 트리의 개수 지정 (트리의 개수가 많을수록 성능이 좋아질 수 있지만 계산 비용 증가) | 100             |
| `criterion`              | 분할 품질을 측정하는 기준 (분류에서는 "gini" 또는 "entropy"를 사용)                 | "gini"          |
| `max_depth`              | 각 트리의 최대 깊이 (설정하지 않으면 트리는 잎 노드가 순수해질 때까지 계속 확장) | None            |
| `min_samples_split`      | 내부 노드를 분할하기 위해 필요한 최소 샘플 수 (과적합 방지 목적)                   | 2               |
| `min_samples_leaf`       | 잎 노드가 되기 위해 필요한 최소 샘플 수 (과적합 방지 목적)                          | 1               |
| `max_features`           | 각 트리를 분할할 때 고려할 최대 특성 수 ()"auto", "sqrt", "log2" 중 선택하거나, 특정 숫자 지정 가능) | "auto"          |
| `bootstrap`              | 각 트리를 만들 때 부트스트랩 샘플링을 사용할지 여부를 결정                               | True            |
| `random_state`           | 결과의 재현성을 위해 난수 시드 고정                                                  | None            |
| `n_jobs`                 | 병렬 계산을 위해 사용할 CPU 코어 수를 지정 (-1로 설정하면 모든 코어를 사용)           | None            |
| `class_weight`           | 각 클래스의 가중치를 자동으로 계산하거나 직접 지정 가능 (불균형 데이터 처리에 유용)    | None            |


In [81]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=100, 
    max_depth=7, 
    random_state=0
)

# 학습
rf_clf.fit(X_train, y_train)

y_pred_train = rf_clf.predict(X_train)
acc_scrore_train = accuracy_score(y_train, y_pred_train)
print('학습 점수: ', acc_score_train)

y_pred_test = rf_clf.predict(X_test)
acc_score_test = accuracy_score(y_test, y_pred_test)
print('테스트 평가 점수: ', acc_score_test)

학습 점수:  0.9859154929577465
테스트 평가 점수:  0.9790209790209791


In [ ]:
# 100개의 decisiontree 확인
# print(rf_clf.estimators_)

# 100개의 decisiontree가 사용한 샘플 데이터 확인
print(len(rf_clf.estimators_samples_)) # 예측기가 100개라 100개로 확인됨

100
